# Massage Crypto Data
### Short notebook used to do some massaging of the crypto-data
Not necessary to use to get this repo to work

In [0]:
import os
import math
import re

import requests

from datetime import datetime
from glob import glob
from collections import defaultdict
from functools import reduce

import numpy as np
import pandas as pd

In [0]:
def is_int(x):
    try:
        int(x)
        return True
    except ValueError:
        return False

In [0]:
# Get all paths and organize per pair
BASE_PATH = '/content/drive/My Drive/NTNU/6. Semester/NEURO140/Data/1M'
paths = glob(BASE_PATH + '/*/*')

pattern = re.compile(f'{BASE_PATH}' + r'/(.*)/(\d{1})\.csv')

d = defaultdict(list)

for s in paths:
    match = pattern.match(s)
    if match is None:
        continue

    pair = match.group(1)
    if pair == 'YNDX':
        continue
        
    d[pair].append(s)

In [0]:
# Get all files per pair and smush them together
total_rows = 0
for pair, paths in d.items():
    print(f'Doing pair {pair}')
    dfs = [pd.read_csv(path) for path in paths if path.endswith('.csv')]
    df = pd.concat(dfs)
    df2 = df.drop_duplicates(['time'])

    total_rows += df2.shape[0]
    
    removed = df.shape[0] - df2.shape[0]

    if removed != 0:
        print(f'{removed} removed')

        for i, frame in enumerate(dfs, start=1):
            frame.to_csv(os.path.join(BASE_PATH, pair, str(i) + ".csv"))
        else:
            split_index = math.floor(df2.shape[0] * 0.8)
            df2.iloc[:split_index].to_csv(os.path.join(BASE_PATH, pair, "1.csv"))
            df2.iloc[split_index:].to_csv(os.path.join(BASE_PATH, pair, "2.csv"))


Doing pair XTZUSD
Doing pair XRPUSD
Doing pair ETHEUR
Doing pair LTCUSD
Doing pair ETHUSD
Doing pair XMRUSD
Doing pair BCHUSD
Doing pair EOSUSD
Doing pair XRPEUR


In [0]:
df3 = df2.copy()
df3['Bbands'] = df3['Bollinger Bands %B']
df3 = df3.drop('Bollinger Bands %B', axis=1)
df3.dropna().head(n=10)

,time,open,high,low,close,VWAP,Volume,Volume MA,Histogram,MACD,Signal,RSI,Bbands
33,1584318780,5267.49,5275.07,5261.67,5269.66,5240.389188,1.960395,3.314851,-1.438625,7.976971,9.415597,48.972961,0.777208
34,1584318840,5264.65,5268.97,5246.71,5249.39,5240.398363,0.152640,3.178103,-2.000454,6.915029,8.915483,44.236913,0.416747
35,1584318900,5267.56,5267.56,5254.82,5254.82,5240.443986,0.596407,2.977628,-1.982482,6.437381,8.419863,45.750428,0.489904
36,1584318960,5259.99,5260.00,5257.49,5260.00,5240.452553,0.111735,2.930103,-1.613479,6.403015,8.016493,47.222086,0.590183
37,1584319020,5257.00,5267.71,5257.00,5257.00,5240.452632,0.000966,2.736221,-1.562151,6.063804,7.625955,46.436407,0.503065
38,1584319080,5255.97,5256.18,5252.39,5252.80,5240.479659,0.495751,2.417107,-1.785648,5.393895,7.179543,45.300058,0.371622
39,1584319140,5267.48,5268.30,5259.31,5259.31,5240.489192,0.106906,2.333525,-1.482127,5.326885,6.809012,47.446746,0.548606
40,1584319200,5267.79,5277.63,5264.32,5264.32,5240.605233,1.008897,2.201196,-0.956540,5.613337,6.569877,49.102221,0.698231
41,1584319260,5269.87,5269.87,5245.87,5245.87,5240.607418,0.040484,2.108562,-1.814301,4.302001,6.116302,43.649141,0.204116
42,1584319320,5258.44,5265.20,5244.32,5254.27,5240.609343,0.033839,1.699474,-1.776513,3.895660,5.672173,46.559069,0.463539


In [0]:
total_rows

183289

In [0]:
url = 'https://api.kraken.com/0/public/OHLC'
params = {
    'pair': 'XBTUSD',
    'interval': 15
}

resp = requests.get(
    url,
    params=params,
    # headers=headers,
)

res = resp.json()

In [0]:
if len(res['error']) > 0:
    print("Error")
    print(res['error'])
else:
    res['result']

    prices = list(res['result'].values())[0]
    last = res['result']['last']
    print(last)

    print(prices)

    df = pd.DataFrame(prices, columns=['time', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'count'])
    df['time'] = pd.to_datetime(df['time'], unit='s')
    
    print(df.head())
    print(df.tail())
    print(df.shape)

1587805200
[[1587159000, '7079.4', '7079.8', '7072.9', '7072.9', '7074.7', '24.16401961', 103], [1587159900, '7072.9', '7072.9', '7034.6', '7048.0', '7048.3', '84.40520045', 201], [1587160800, '7050.1', '7065.0', '7050.1', '7061.9', '7056.8', '144.35057911', 155], [1587161700, '7064.4', '7066.0', '7060.9', '7064.1', '7065.0', '35.76623440', 155], [1587162600, '7064.1', '7069.7', '7064.1', '7067.0', '7066.0', '29.60312726', 97], [1587163500, '7066.9', '7066.9', '7053.7', '7055.6', '7061.6', '18.76789755', 114], [1587164400, '7066.3', '7076.6', '7062.1', '7074.5', '7068.3', '19.25726589', 67], [1587165300, '7074.5', '7153.5', '7071.7', '7145.2', '7118.9', '225.91496126', 591], [1587166200, '7141.5', '7157.8', '7029.7', '7061.8', '7092.3', '142.48928948', 417], [1587167100, '7060.5', '7074.0', '7024.7', '7034.1', '7046.4', '129.48139481', 268], [1587168000, '7034.5', '7059.0', '7025.4', '7059.0', '7049.5', '77.65550794', 179], [1587168900, '7058.9', '7067.6', '7050.9', '7055.0', '7058.8',

In [0]:
url = 'https://api.kraken.com/0/public/Spread'
params = {
    'pair': 'XBTUSD',
}

resp = requests.get(
    url,
    params=params,
    # headers=headers,
)

res = resp.json()

{'error': [], 'result': {'XXBTZUSD': [[1587804528, '7524.40000', '7527.00000'], [1587804528, '7524.40000', '7526.70000'], [1587804528, '7524.40000', '7526.40000'], [1587804528, '7524.40000', '7526.20000'], [1587804529, '7523.60000', '7526.20000'], [1587804530, '7525.20000', '7526.00000'], [1587804531, '7525.20000', '7526.20000'], [1587804531, '7525.20000', '7526.30000'], [1587804531, '7525.20000', '7526.00000'], [1587804532, '7525.20000', '7525.80000'], [1587804532, '7525.20000', '7525.50000'], [1587804536, '7525.20000', '7525.40000'], [1587804536, '7525.20000', '7525.30000'], [1587804541, '7525.20000', '7525.40000'], [1587804543, '7525.30000', '7525.40000'], [1587804543, '7525.70000', '7526.10000'], [1587804543, '7526.00000', '7526.10000'], [1587804543, '7526.60000', '7529.20000'], [1587804544, '7526.70000', '7529.20000'], [1587804544, '7526.90000', '7529.20000'], [1587804544, '7529.10000', '7529.20000'], [1587804546, '7529.10000', '7531.80000'], [1587804546, '7529.40000', '7531.80000

In [0]:
if len(res['error']) > 0:
    print("Error")
    print(res['error'])
else:
    res['result']

    prices = list(res['result'].values())[0]
    last = res['result']['last']
    print(last)

    print(prices)

    df = pd.DataFrame(prices, columns=['time', 'bid', 'ask'])
    df['time_format'] = pd.to_datetime(df['time'], unit='s')
    
    print(df.head())
    print(df.tail())
    print(df.shape)

1587805573
[[1587804528, '7524.40000', '7527.00000'], [1587804528, '7524.40000', '7526.70000'], [1587804528, '7524.40000', '7526.40000'], [1587804528, '7524.40000', '7526.20000'], [1587804529, '7523.60000', '7526.20000'], [1587804530, '7525.20000', '7526.00000'], [1587804531, '7525.20000', '7526.20000'], [1587804531, '7525.20000', '7526.30000'], [1587804531, '7525.20000', '7526.00000'], [1587804532, '7525.20000', '7525.80000'], [1587804532, '7525.20000', '7525.50000'], [1587804536, '7525.20000', '7525.40000'], [1587804536, '7525.20000', '7525.30000'], [1587804541, '7525.20000', '7525.40000'], [1587804543, '7525.30000', '7525.40000'], [1587804543, '7525.70000', '7526.10000'], [1587804543, '7526.00000', '7526.10000'], [1587804543, '7526.60000', '7529.20000'], [1587804544, '7526.70000', '7529.20000'], [1587804544, '7526.90000', '7529.20000'], [1587804544, '7529.10000', '7529.20000'], [1587804546, '7529.10000', '7531.80000'], [1587804546, '7529.40000', '7531.80000'], [1587804546, '7530.700

In [0]:
# Test rate limiting
url = 'https://api.kraken.com/0/public/OHLC'
params = {
    'pair': 'XBTUSD',
}

for _ in range(500):
    resp = requests.get(
        url,
        params=params,
        # headers=headers,
    )

    print(resp.json()['error'], list(resp.json()['result'].keys()))

[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 'last']
[] ['XXBTZUSD', 

In [2]:
df_eth1 = pd.read_csv('/content/drive/My Drive/NTNU/6. Semester/NEURO140/Data/1M/ETHEUR/1.csv')
df_eth2 = pd.read_csv('/content/drive/My Drive/NTNU/6. Semester/NEURO140/Data/1M/ETHEUR/2.csv')

print(np.min(df_eth1['time']), np.max(df_eth1['time']))
print(np.min(df_eth2['time']), np.max(df_eth2['time']))

df_eth = pd.concat([df_eth1, df_eth2])
df_eth_nodup = df_eth.drop_duplicates(['time'])

df_eth_nodup.to_csv('/content/drive/My Drive/NTNU/6. Semester/NEURO140/Data/1M/ETHEUR/1+2.csv')

1584316800 1585784400
1585784460 1586161140
